# XE Candidate Datastore Demo

This sample **doesn't attempt to lock or unlock datastores**, and thus assumes singular access.

In [1]:
HOST = '127.0.0.1'
PORT_NC = 2223
USER = 'vagrant'
PASS = 'vagrant'

## Connect ncclient

In [2]:
from ncclient import manager
from lxml import etree


def pretty_print(retval):
    print(etree.tostring(retval.data, pretty_print=True))

def my_unknown_host_cb(host, fingerprint):
    return True

m = manager.connect(host=HOST, port=PORT_NC, username=USER, password=PASS,
                    allow_agent=False,
                    look_for_keys=False,
                    hostkey_verify=False,
                    unknown_host_cb=my_unknown_host_cb)

## Loopback Addition Sample Edit Config

In [3]:
config = '''<config>
  <native xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-native">
    <interface>
      <Loopback>
        <name>101</name>
        <description>hello</description>
      </Loopback>
    </interface>
  </native>
</config>
'''

In [4]:
running_before = m.get_config(source='running', filter=('xpath', '/native')).data

In [5]:
m.edit_config(config, format='xml', target='candidate', default_operation='merge')

<?xml version="1.0" encoding="UTF-8"?>
<rpc-reply xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:3662503e-660c-4bb2-a0e4-41a50c39d62c" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0"><ok/></rpc-reply>

In [6]:
running_after = m.get_config(source='running', filter=('xpath', '/native')).data

In [7]:
candidate_after = m.get_config(source='candidate', filter=('xpath', '/native')).data

### Display An XML Diff In Text Form

In [8]:
from difflib import context_diff

running_before_xml = etree.tostring(running_before, pretty_print=True)
running_after_xml = etree.tostring(running_after, pretty_print=True)
candidate_after_xml = etree.tostring(candidate_after, pretty_print=True)

#
# remember to skip the first few lines that have timestamps & stuff that may differ
#
print('\n'.join(context_diff(running_after_xml.decode().splitlines(),
                             candidate_after_xml.decode().splitlines())))

*** 

--- 

***************

*** 145,150 ****

--- 145,154 ----

            <auto>true</auto>
          </negotiation>
        </GigabitEthernet>
+       <Loopback>
+         <name>101</name>
+         <description>hello</description>
+       </Loopback>
      </interface>
      <control-plane/>
      <multilink>


### Commit Or Discard?

In [ ]:
m.commit()

In [9]:
m.discard_changes()

<?xml version="1.0" encoding="UTF-8"?>
<rpc-reply xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:e9177690-517b-4bf4-8eb9-675eff17c7bd" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0"><ok/></rpc-reply>

# BGP Remove Sample Edit Config

In [10]:
config = '''<config xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0">
  <native xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-native">
    <router>
      <bgp xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-bgp" nc:operation="remove">
        <id>101</id>
      </bgp>
    </router>
  </native>
</config>
'''

In [11]:
running_before = m.get_config(source='running', filter=('xpath', '/native')).data

In [12]:
m.edit_config(config, format='xml', target='candidate', default_operation='merge')

<?xml version="1.0" encoding="UTF-8"?>
<rpc-reply xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:7bba143b-99e8-4936-8b6e-7b64441383e1" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0"><ok/></rpc-reply>

In [13]:
running_after = m.get_config(source='running', filter=('xpath', '/native')).data

In [14]:
candidate_after = m.get_config(source='candidate', filter=('xpath', '/native')).data

### Display An XML Diff In Text Form

In [15]:
from difflib import context_diff

running_before_xml = etree.tostring(running_before, pretty_print=True)
running_after_xml = etree.tostring(running_after, pretty_print=True)
candidate_after_xml = etree.tostring(candidate_after, pretty_print=True)

#
# remember to skip the first few lines that have timestamps & stuff that may differ
#
print('\n'.join(context_diff(running_after_xml.decode().splitlines(),
                             candidate_after_xml.decode().splitlines())))

*** 

--- 

***************

*** 183,216 ****

          </trustpoint>
        </pki>
      </crypto>
-     <router>
-       <bgp xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-bgp">
-         <id>101</id>
-         <bgp>
-           <router-id>172.16.1.1</router-id>
-           <log-neighbor-changes/>
-         </bgp>
-         <neighbor>
-           <id>10.10.10.2</id>
-           <remote-as>102</remote-as>
-         </neighbor>
-         <address-family>
-           <no-vrf>
-             <ipv4>
-               <af-name>unicast</af-name>
-               <neighbor>
-                 <id>10.10.10.2</id>
-                 <activate/>
-               </neighbor>
-               <network>
-                 <number>172.18.1.0</number>
-                 <mask>255.255.255.0</mask>
-               </network>
-             </ipv4>
-           </no-vrf>
-         </address-family>
-       </bgp>
-     </router>
      <license>
        <udi>
          <pid>CSR1000V</pid>
--- 183,188 ----



### Commit Or Discard?

In [ ]:
m.commit()

In [16]:
m.discard_changes()

<?xml version="1.0" encoding="UTF-8"?>
<rpc-reply xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:f9f2d6f0-3c1a-4f86-ae4a-4fd40d6b8eb2" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0"><ok/></rpc-reply>

## Diff Before & After (XML)

## Tidyup Sessions

In [17]:
m.close_session()

<?xml version="1.0" encoding="UTF-8"?>
<rpc-reply xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:cdf2c970-f35e-4afd-83ee-014eb91f3e1d" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0"><ok/></rpc-reply>